# 2D case - Figure 15
Two-parameter 2D problem - Investigation of  the  convergence  of  the  reduced-order model and of the evolution of the size of the reduced-order basis.

## Libraries import  

In [1]:

import sys  
import torch
import torch.nn as nn
from neurom.HiDeNN_PDE import MeshNN, NeuROM, MeshNN_2D, MeshNN_1D
import neurom.src.Pre_processing as pre
from neurom.src.PDE_Library import Strain, Stress,VonMises_plain_strain
from neurom.src.Training import Training_NeuROM_multi_level
import neurom.Post.Plots as Pplot
import time
import os
import torch._dynamo as dynamo
from importlib import reload  
import tomllib
import numpy as np
import argparse

torch.manual_seed(0)

* WARNING: could not load tikzplotlib


## Load the config file

In [2]:
    Configuration_file = 'Configurations/config_2D_ROM.toml'

    with open(Configuration_file, mode="rb") as file:
        config = tomllib.load(file)

## Definition of the space domain and mechanical proprieties of the structure

The initial Material parameters, the geometry, mesh and the boundary conditions are set. 

In [3]:

# Material parameters definition

Mat = pre.Material(             flag_lame = False,                                  # If True should input lmbda and mu instead of E and nu
                                coef1     = config["material"]["E"],                # Young Modulus
                                coef2     = config["material"]["nu"]                # Poisson's ratio
                )


# Create mesh object
MaxElemSize = pre.ElementSize(
                                dimension     = config["interpolation"]["dimension"],
                                L             = config["geometry"]["L"],
                                order         = config["interpolation"]["order"],
                                np            = config["interpolation"]["np"],
                                MaxElemSize2D = config["interpolation"]["MaxElemSize2D"]
                            )
Excluded = []
Mesh_object = pre.Mesh( 
                                config["geometry"]["Name"],                         # Create the mesh object
                                MaxElemSize, 
                                config["interpolation"]["order"], 
                                config["interpolation"]["dimension"]
                        )

Mesh_object.AddBorders(config["Borders"]["Borders"])
Mesh_object.AddBCs(                                                                 # Include Boundary physical domains infos (BCs+volume)
                                config["geometry"]["Volume_element"],
                                Excluded,
                                config["DirichletDictionryList"]
                    )                   

Mesh_object.MeshGeo()                                                               # Mesh the .geo file if .msh does not exist
Mesh_object.ReadMesh()       
Mesh_object.ExportMeshVtk()

 
 
   _   _            ____   ___  __  __ 
  | \ | | ___ _   _|  _ \ / _ \|  \/  |
  |  \| |/ _ \ | | | |_) | | | | |\/| |
  | |\  |  __/ |_| |  _ <| |_| | |  | |
  |_| \_|\___|\__,_|_| \_\ ___/|_|  |_|

                  unknown version

************ MESH READING COMPLETE ************

 * Dimension of the problem: 2D
 * Elements type:            t3: 3-node triangle
 * Number of Dofs:           224



fatal: No names found, cannot describe anything.


## Parametric study definition

The hypercube describing the parametric domain used for the tensor decomposition is set-up here

In [4]:
ParameterHypercube = torch.tensor([ [   config["parameters"]["para_1_min"],
                                        config["parameters"]["para_1_max"],
                                        config["parameters"]["N_para_1"]],
                                    [   config["parameters"]["para_2_min"],
                                        config["parameters"]["para_2_max"],
                                        config["parameters"]["N_para_2"]]])

## Initialisation of the surrogate model

In [5]:
ROM_model = NeuROM(                                                                 # Build the surrogate (reduced-order) model
                    Mesh_object, 
                    ParameterHypercube, 
                    config,
                    config["solver"]["n_modes_ini"],
                    config["solver"]["n_modes_max"]
                    )

## Training the model

In [6]:
ROM_model.Freeze_Mesh()                                                             # Set space mesh coordinates as untrainable
ROM_model.Freeze_MeshPara()                                                         # Set parameters mesh coordinates as untrainable

ROM_model.TrainingParameters(   
                                loss_decrease_c = config["training"]["loss_decrease_c"], 
                                Max_epochs = config["training"]["n_epochs"], 
                                learning_rate = config["training"]["learning_rate"]
                            )

ROM_model.train()                                                                   # Put the model in training mode
ROM_model, Mesh_object = Training_NeuROM_multi_level(ROM_model,config, Mat)         


* Refinement level: 0

**************** START TRAINING ***************

epoch 100 loss = -3.27861e-04 modes = 1
epoch 200 loss = -1.25096e-03 modes = 1
epoch 300 loss = -2.29776e-03 modes = 1
epoch 400 loss = -3.44243e-03 modes = 1
epoch 500 loss = -4.01850e-03 modes = 1
epoch 600 loss = -4.17413e-03 modes = 1
epoch 700 loss = -4.20585e-03 modes = 1
epoch 800 loss = -4.23706e-03 modes = 2
epoch 900 loss = -4.78608e-03 modes = 2
epoch 1000 loss = -5.07869e-03 modes = 2
epoch 1100 loss = -5.22825e-03 modes = 2
epoch 1200 loss = -5.33641e-03 modes = 2
epoch 1300 loss = -5.37339e-03 modes = 2
*************** END FIRST PHASE ***************

* Training time: 5.033659219741821s
* Saving time: 1.9073486328125e-06s
* Evaluation time: 2.802030563354492s
* Backward time: 2.012726068496704s
* Update time: 0.16192030906677246s
* Average epoch time: 0.0037620771447995677s
************** START SECOND PAHSE *************

epoch 5 loss = -5.377e-03
*************** END OF TRAINING ***************

* Tr

## Plotting area

Reproducing figure 15

In [7]:
tikz = False

import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator

Pplot.Plot_2D_PyVista(ROM_model, 
                Mesh_object, 
                config, 
                E = config["postprocess"]["Default_E"], 
                theta = config["postprocess"]["Default_theta"], 
                scalar_field_name = config["postprocess"]["scalar_field_name"], 
                scaling_factor = config["postprocess"]["scaling_factor"], 
                Interactive_parameter = config["postprocess"]["Interactive_parameter"],
                Plot_mesh = config["postprocess"]["Plot_mesh"],
                color_map = config["postprocess"]["colormap"])


************** Available commands *************

* Take a screenshot : s

* Reset the camera  : r



Widget(value='<iframe src="http://localhost:54363/index.html?ui=P_0x151f269f0_0&reconnect=auto" class="pyvista…